In this tutorial, we will see how we can train a model to classify text (here based on their sentiment). First we will see how to do this quickly in a few lines of code, then how to get state-of-the art results using the approach of the ULMFit paper.

We will use the IMDb dataset from the paper Learning Word Vectors for Sentiment Analysis, containing a few thousand movie reviews.
https://ai.stanford.edu/~amaas/data/sentiment/

But is too heavy so instead we choose another more simple (IMB_SAMPLE) that is only a csv file with a lot of opinions

Para ejecutar este proyecto hemos tenido que cambiar un poco la configuracion del proyecto. En primer lugar tuvimos que instalar miniconda 3 https://www.anaconda.com/download/success

y luego configurarlo para que utilizara nuestra gpu ( cuando trabajamos con texto  es la unica manera de obtener algo mas o menos plausible)
en el directorio encontraras un enviroment.yml 

conda env create -f environment.yml
conda activate pytorch

Comprobamos que estamos usando Anaconda y la version 3.12 de python 
Para usar torch neceistamos python 3.12 sino revienta !!!!!

In [4]:
import sys
print(sys.version)

3.12.12 | packaged by Anaconda, Inc. | (main, Oct 14 2025, 16:10:16) [MSC v.1929 64 bit (AMD64)]


In [ ]:

import torch
print("GPU disponible:", torch.cuda.is_available())
print("Dispositivo:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")

print("Versión PyTorch:", torch.__version__)
print("Versión CUDA de PyTorch:", torch.version.cuda)
print("CUDA disponible:", torch.cuda.is_available())


GPU disponible: True
Dispositivo: NVIDIA GeForce RTX 4060 Laptop GPU
Versión PyTorch: 2.5.1
Versión CUDA de PyTorch: 12.1
CUDA disponible: True


In [1]:
from fastai.text.all import *

We can download the data and decompress it with the following command:

In [2]:
path = untar_data(URLs.IMDB_SAMPLE)
path.ls()


(#1) [Path('C:/Users/tonid/.fastai/data/imdb_sample/texts.csv')]

La informacion aparece unicamente en un fichero csv donde la columna label nos informa si es correcta o no la informacion

In [3]:

path = untar_data(URLs.IMDB_SAMPLE)
dls = TextDataLoaders.from_csv(
    path=path,
    csv_fname='texts.csv',
    text_col='text',
    label_col='label',
    valid_pct=0.2,
    bs=64  # batch size
)

print("Clases encontradas:", dls.vocab)
dls.show_batch(max_n=2)


Due to IPython and Windows limitation, python multiprocessing isn't available now.
So `n_workers` has to be changed to 0 to avoid getting stuck
Clases encontradas: [['xxunk', 'xxpad', 'xxbos', 'xxeos', 'xxfld', 'xxrep', 'xxwrep', 'xxup', 'xxmaj', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is', 'it', 'in', 'i', 'that', 'this', '"', "'s", '-', 'was', '\n\n', 'as', 'for', 'movie', 'with', 'but', 'film', 'you', ')', 'on', '(', "n't", 'are', 'not', 'he', 'his', 'have', 'be', 'one', 'they', 'all', 'at', 'by', 'an', 'like', 'from', 'so', 'who', '!', 'there', 'about', 'or', 'just', 'if', 'out', 'do', 'her', 'some', 'what', 'has', "'", 'more', 'good', '?', 'very', 'up', 'when', 'she', 'would', 'no', 'were', 'their', 'even', 'time', 'really', 'only', 'my', 'can', 'story', 'had', 'which', '…', 'see', 'did', 'me', 'we', 'well', 'into', 'does', ':', 'will', 'bad', 'than', 'also', 'get', 'much', 'people', 'because', 'could', 'other', 'first', 'been', 'how', 'any', 'its', 'most', 'then', 'great', 'mak

,text,category
0,"xxbos xxup the xxup shop xxup around xxup the xxup corner is one of the xxunk and most feel - good romantic comedies ever made . xxmaj there 's just no getting around that , and it 's hard to actually put one 's feeling for this film into words . xxmaj it 's not one of those films that tries too hard , nor does it come up with the xxunk possible scenarios to get the two protagonists together in the end . xxmaj in fact , all its charm is xxunk , contained within the characters and the setting and the plot … which is highly believable to xxunk . xxmaj it 's easy to think that such a love story , as beautiful as any other ever told , * could * happen to you … a feeling you do n't often get from other romantic comedies",positive
1,"xxbos xxmaj now that xxmaj che(2008 ) has finished its relatively short xxmaj australian cinema run ( extremely limited xxunk screen in xxmaj xxunk , after xxunk ) , i can xxunk join both xxunk of "" at xxmaj the xxmaj movies "" in taking xxmaj steven xxmaj soderbergh to task . \n\n xxmaj it 's usually satisfying to watch a film director change his style / subject , but xxmaj soderbergh 's most recent stinker , xxmaj the xxmaj girlfriend xxmaj xxunk ) , was also missing a story , so narrative ( and editing ? ) seem to suddenly be xxmaj soderbergh 's main challenge . xxmaj strange , after 20 - odd years in the business . xxmaj he was probably never much good at narrative , just xxunk it well inside "" edgy "" projects . \n\n xxmaj none of this excuses him this present ,",negative


In [6]:
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)
learn.fine_tune(4, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.618220,0.609347,0.690000,00:02


epoch,train_loss,valid_loss,accuracy,time
0,0.426003,0.532202,0.735000,00:07
1,0.372444,0.426498,0.810000,00:11
2,0.303386,0.411257,0.805000,00:12
3,0.248246,0.392918,0.830000,00:12


In [7]:
learn.predict("This movie was absolutely terrible and boring")


('negative', tensor(0), tensor([0.9684, 0.0316]))

In [8]:
learn.predict("Amazing film! Best movie I've ever seen!")

('positive', tensor(1), tensor([0.0277, 0.9723]))

In [9]:
learn.predict("I hate this movie. It was a waste of time.")

('negative', tensor(0), tensor([0.9361, 0.0639]))